In [1]:
# Google

import pandas as pd
import numpy as np
import datetime
import os
import glob

# 19-48 Google

In [2]:
list_file_19_48=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/*.csv")
list_file_19_48=[x for x in list_file_19_48 if "1948" in x]
print(len(list_file_19_48))

# test only
list_file_19_48=[x for x in list_file_19_48 if os.path.basename(x)[0]=="T"]
print(len(list_file_19_48))
list_file_19_48_T_SOTF=[x for x in list_file_19_48 if x.split("ted_segment_files/T_")[1][:4]=="SOTF"]
print(len(list_file_19_48_T_SOTF))
list_file_19_48_T_SOTF

16
8
4


['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureLow_RFM20Q1.csv']

In [3]:
import hashlib
import uuid
# Import appropriate modules from the client library.
from googleads import adwords
import os
import datetime
auth_file_path = '/home/jian/Docs/Google_Ads_Access/googleads_BL_99xBL2H.yaml'

def upload_and_return_count_in_chunk(service, user_list_id, email_list):
    ids_in_chunk=0
    mutate_members_operation = {
      'operand': {
          'userListId': user_list_id,
          'membersList': email_list
      },
      'operator': 'ADD'
    }
    response = service.mutateMembers([mutate_members_operation])
    if 'userLists' in response:
        for user_list in response['userLists']:
            # print('Uploaded %i items to user list with name "%s" and ID "%d"'
            #     % (len(email_list), user_list['name'], user_list['id']))

            
            ids_in_chunk+=len(email_list)
    return ids_in_chunk
            
def run_the_uploading(client, email_file_path):
  # Initialize appropriate services.
    total_ids_in_seg=0
    user_list_service = client.GetService('AdwordsUserListService', 'v201809')
    file_base_name=os.path.basename(email_file_path).split(".")[0]
    user_list = {
        'xsi_type': 'CrmBasedUserList',
        'name': file_base_name,
        'description': 'A list of customers that originated from email addresses as '+file_base_name,
        # CRM-based user lists can use a membershipLifeSpan of 10000 to indicate
        # unlimited; otherwise normal values apply.
        'membershipLifeSpan': 10000,
        'uploadKeyType': 'CONTACT_INFO'
     }
    # Create an operation to add the user list.
    operations = [{
        'operator': 'ADD',
        'operand': user_list
    }]
    result = user_list_service.mutate(operations)
    user_list_id = result['value'][0]['id']

    df = pd.read_csv(email_file_path, chunksize=3000)
    for i in df:
        mail_col = [k for k in i.columns if 'email' in k.lower()][0]
        new_col = ['email' if h == mail_col else h for h in i.columns.tolist()]
        i.columns = new_col
        emails = i['email']
        emails = set([i for i in emails if str(i) != 'nan'])
        members = [{'hashedEmail': email} for email in emails]
        ids_in_chunk=upload_and_return_count_in_chunk(user_list_service, user_list_id, members)
        
        total_ids_in_seg+=ids_in_chunk
    print(datetime.datetime.now(),"all %s ids uploaded in the segment "%str(total_ids_in_seg))


In [4]:
BL_client = adwords.AdWordsClient.LoadFromStorage(auth_file_path)
for local_seg_file in list_file_19_48_T_SOTF:
    run_the_uploading(BL_client, local_seg_file)

2020-03-05 15:15:49.314414 all 935349 ids uploaded in the segment 
2020-03-05 15:16:29.780513 all 180867 ids uploaded in the segment 
2020-03-05 15:21:10.025669 all 419158 ids uploaded in the segment 
2020-03-05 15:22:58.072689 all 85333 ids uploaded in the segment 


In [5]:
list_file_19_48_T_SOTF

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureLow_RFM20Q1.csv']